In [50]:
# iPython Setting
%load_ext autoreload
%autoreload 2

from datamodules import TableDatasetModule
from models import get_model
import torch
from pytorch_lightning import Trainer
import pandas as pd

from shutil import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
path_to_ckpt = "../inference-checkpoints/best-fasterrcnn-v2.ckpt"

model_name = "VanillaFasterRCNNV2"

In [42]:
vanilla_model = get_model(model_name=model_name)
    
# Check if the correct model name was specified
if vanilla_model is None:
    print("Model with the name: " + model_name + " not found.")
    

# Load the model with the checkpoint weights
model = vanilla_model.load_from_checkpoint(path_to_ckpt)

# Move the model to appropriate storage i.e. RAM if only CPU is
# available and VRAM if only GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Loaded Successfully")

Testing DataLoader 0:   2%|▏         | 5/227 [31:43<23:28:52, 380.78s/it]
Loaded Successfully


In [36]:
path_to_dataset = "../datasets/cellmodules-2/csv/"

dm = TableDatasetModule(
    path=path_to_dataset,
    num_workers=2,
    batch_size=2
)

dm.prepare_data()
dm.setup(stage='test')

dl = dm.test_dataloader()


Dataset Information
Total Images in Dataset: 453
Number of Training Images: 362
Number of Evaluation Images: 91


TESTING
TESTING
TESTING



In [45]:
trainer = Trainer(
    max_epochs=50,
    accelerator='auto',
    devices='auto',
    log_every_n_steps=1
    )

results = trainer.test(
    model=model,
    datamodule=dm
)

print(results)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/anaconda3/envs/Thesis/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,



Dataset Information
Total Images in Dataset: 4
Number of Training Images: 3
Number of Evaluation Images: 1


TESTING
TESTING
TESTING

Testing DataLoader 0: 100%|██████████| 2/2 [00:14<00:00,  7.41s/it]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    val/epoch/avg_iou       0.7325873374938965
val/epoch/precision_75_50           1.0
val/epoch/precision_75_75           1.0
val/epoch/precision_75_90   0.8181818181818182
 val/epoch/recall_75_50             1.0
 val/epoch/recall_75_75             1.0
 val/epoch/recall_75_90     0.8181818181818182
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[{'val/epoch/avg_iou': 0.7325873374938965, 'val/epoch/precision_75_50': 1.0, 'val/epoch/p

In [33]:
all_annotations_path = "../datasets/cellmodules-1/csv/annotations.csv"
sup_eval_path = "supposedly_eval.csv"

all_annotations_df = pd.read_csv(all_annotations_path, names=['image', 'x1', 'y1', 'x2', 'y2', 'class'])
sup_eval_df = pd.read_csv(sup_eval_path, names=['image'])

sup_eval_annot = sup_eval_df.merge(right=all_annotations_df)
sup_eval_annot['image'].nunique()

sup_eval_annot.to_csv("annotations-eval.csv", index=False, header=False)


In [51]:
filelist = sup_eval_df['image'].values.tolist()

path_to_src_folder = "../../Evaluation-Data/detection/data/images/"
path_to_dst_folder = "../../Evaluation-Data/detection/data/images/required/"

for i, file in enumerate(filelist):
    
    copy(
        src=path_to_src_folder + str(file),
        dst=path_to_dst_folder + str(file)
    )

    print("Copied File: {}".format(i+1))

Copied File: 1
Copied File: 2
Copied File: 3
Copied File: 4
Copied File: 5
Copied File: 6
Copied File: 7
Copied File: 8
Copied File: 9
Copied File: 10
Copied File: 11
Copied File: 12
Copied File: 13
Copied File: 14
Copied File: 15
Copied File: 16
Copied File: 17
Copied File: 18
Copied File: 19
Copied File: 20
Copied File: 21
Copied File: 22
Copied File: 23
Copied File: 24
Copied File: 25
Copied File: 26
Copied File: 27
Copied File: 28
Copied File: 29
Copied File: 30
Copied File: 31
Copied File: 32
Copied File: 33
Copied File: 34
Copied File: 35
Copied File: 36
Copied File: 37
Copied File: 38
Copied File: 39
Copied File: 40
Copied File: 41
Copied File: 42
Copied File: 43
Copied File: 44
Copied File: 45
Copied File: 46
Copied File: 47
Copied File: 48
Copied File: 49
Copied File: 50
Copied File: 51
Copied File: 52
Copied File: 53
Copied File: 54
Copied File: 55
Copied File: 56
Copied File: 57
Copied File: 58
Copied File: 59
Copied File: 60
Copied File: 61
Copied File: 62
Copied File: 63
C